In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import subprocess

## Utils

In [ ]:
# lambdas
file_name = lambda tree_type: f'results/{tree_type}.csv'
get_result = lambda tree_type: pd.read_csv(file_name(tree_type))

In [ ]:
# constant variables
K = 20

TREE_TYPES = ['bst', 'rb', 'splay']
N_RANGE = np.arange(1, 11) * 10000
GENERATE = ['ascending', 'random']

In [ ]:
def execute(tree_type, n, generate):
    subprocess.run(f'./trees -t {tree_type} -n {n} -g {generate}')
v_execute = np.vectorize(execute)

def run(tree_type, k):
    for gen in GENERATE:
        print(f'\t-> {gen}')
        for n in N_RANGE:
            T = np.full(shape=(k,), fill_value=tree_type)
            N = np.full(shape=(k,), fill_value=n)
            G = np.full(shape=(k,), fill_value=gen)
            v_execute(T, N, G)


def process_result(df):
    def _process_gen(df_gen, case):
        df_gen = df_gen[df_gen['case'] == case]
        df_cmp = pd.DataFrame(df_gen.groupby('n', as_index=False)['comparisons'].mean())
        df_ptr = pd.DataFrame(df_gen.groupby('n', as_index=False)['ptr_ops'].mean())
        df_h = pd.DataFrame(df_gen.groupby('n', as_index=False)['height'].mean())
        df_gen = df_cmp.merge(df_ptr, on='n', how='left').merge(df_h, on='n', how='left')
        return df_gen

    df_asc = df[df['gen'] == 'ascending']
    df_rand = df[df['gen'] == 'random']

    return {
        'asc_avg': {
            'name': 'gen = ascending, case = avg',
            'df': _process_gen(df_asc, 'avg')
        },
        'asc_worst': {
            'name': 'gen = ascending, case = worst',
            'df': _process_gen(df_asc, 'worst')
        },
        'rand_avg': {
            'name': 'gen = random, case = avg',
            'df': _process_gen(df_rand, 'avg')
        },
        'rand_worst': {
            'name': 'gen = random, case = worst',
            'df': _process_gen(df_rand, 'worst')
        }
    }



def plot_results(results: dict):
    fig = plt.figure(figsize=(16, 10))
    axs = [plt.subplot2grid((4,3), (i % 4, int(i / 4)), colspan=1, rowspan=1) for i in range(12)]
    plt.subplots_adjust(hspace=0.2, wspace=0.25)

    axs[0].set_title("Comparison count")
    axs[4].set_title("Pointer operation count")
    axs[8].set_title("Tree height")

    axs[0].set_ylabel("Gen - asc; case - avg")
    axs[1].set_ylabel("Gen - asc; case - worst")
    axs[2].set_ylabel("Gen - rand; case - avg")
    axs[3].set_ylabel("Gen - rand; case - worst")


    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for (i, (tree, res_dict)) in enumerate(results.items()):
        axs[0].plot(
            res_dict['asc_avg']['df']['n'], 
            res_dict['asc_avg']['df']['comparisons'],
            color=colors[i], linestyle='-', label=tree
        )
        axs[1].plot(
            res_dict['asc_worst']['df']['n'], 
            res_dict['asc_worst']['df']['comparisons'],
            color=colors[i], linestyle=':', label=tree
        )
        axs[2].plot(
            res_dict['rand_avg']['df']['n'], 
            res_dict['rand_avg']['df']['comparisons'],
            color=colors[i], linestyle='-', label=tree
        )
        axs[3].plot(
            res_dict['rand_worst']['df']['n'], 
            res_dict['rand_worst']['df']['comparisons'],
            color=colors[i], linestyle=':', label=tree
        )

        axs[4].plot(
            res_dict['asc_avg']['df']['n'], 
            res_dict['asc_avg']['df']['ptr_ops'],
            color=colors[i], linestyle='-', label=tree
        )
        axs[5].plot(
            res_dict['asc_worst']['df']['n'], 
            res_dict['asc_worst']['df']['ptr_ops'],
            color=colors[i], linestyle=':', label=tree
        )
        axs[6].plot(
            res_dict['rand_avg']['df']['n'], 
            res_dict['rand_avg']['df']['ptr_ops'],
            color=colors[i], linestyle='-', label=tree
        )
        axs[7].plot(
            res_dict['rand_worst']['df']['n'], 
            res_dict['rand_worst']['df']['ptr_ops'],
            color=colors[i], linestyle=':', label=tree
        )

        axs[8].plot(
            res_dict['asc_avg']['df']['n'], 
            res_dict['asc_avg']['df']['height'],
            color=colors[i], linestyle='-', label=tree
        )
        axs[9].plot(
            res_dict['asc_worst']['df']['n'], 
            res_dict['asc_worst']['df']['height'],
            color=colors[i], linestyle=':', label=tree
        )
        axs[10].plot(
            res_dict['rand_avg']['df']['n'], 
            res_dict['rand_avg']['df']['height'],
            color=colors[i], linestyle='-', label=tree
        )
        axs[11].plot(
            res_dict['rand_worst']['df']['n'], 
            res_dict['rand_worst']['df']['height'],
            color=colors[i], linestyle=':', label=tree
        )

    for i in range(len(axs)):
        axs[i].set_xlabel("$n$ - tree size")
        axs[i].legend(loc=2)

    plt.show();

## Init necessary files

In [ ]:
# Init result .csv files
for type in TREE_TYPES:
    df = pd.DataFrame(columns=['n', 'gen', 'case', 'comparisons', 'ptr_ops', 'height'])
    df.to_csv(file_name(type), index=False)

In [ ]:
subprocess.run('make clean'.split())
subprocess.run('make all'.split())

# Run experiments

In [ ]:
print('executing:')
for type in ['bst', 'splay', 'rb']:
    print(f' -> {type}')
    run(type, K)

In [ ]:
results = {t: get_result(t) for t in TREE_TYPES}
results = {t : process_result(results[t]) for t in results.keys()}

In [ ]:
plot_results(results)